In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import animation

# ===============================
# Parâmetros do modelo
# ===============================

n = 20                       # tamanho da grade (NxN)
proporcao_inicial = 0.2      # fração inicial de alunos atentos
theta = 3                    # limiar de influência social
Ts = 5                       # tempo de saturação
pd = 0.3                     # probabilidade de reinício
iteracoes = 20               # número de iterações

# Professor
raio_professor = 2           # raio de influência (Moore estendida)
passos_professor = [(-1,0), (1,0), (0,-1), (0,1)]

# ===============================
# Inicialização da grade
# ===============================

# Estados: 0 = desatenção | 1 = atenção | 2 = saturação
grade = np.zeros((n, n), dtype=int)

# Contador de tempo de atenção (τ)
tau = np.zeros((n, n), dtype=int)

# Inicializar alunos atentos
num_estado_1 = int(proporcao_inicial * n * n)
indices = np.random.choice(n * n, num_estado_1, replace=False)

for idx in indices:
    i = idx // n
    j = idx % n
    grade[i, j] = 1

# Inicializar posição do professor
prof_i, prof_j = n // 2, n // 2

In [ ]:
# ===============================
# Função de vizinhança de Moore
# ===============================

def contar_atentos(grade, i, j):
    contagem = 0
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            if di == 0 and dj == 0:
                continue
            ni, nj = i + di, j + dj
            if 0 <= ni < n and 0 <= nj < n:
                if grade[ni, nj] == 1:
                    contagem += 1
    return contagem


# ===============================
# Estruturas de registro
# ===============================

historico_0 = []
historico_1 = []
historico_2 = []
historico_grades = []
historico_prof = []

# ===============================
# Loop principal do autômato
# ===============================

for t in range(iteracoes):
    nova_grade = grade.copy()

    # ---------------------------
    # Movimento do professor
    # ---------------------------
    di, dj = random.choice(passos_professor)
    prof_i = min(max(prof_i + di, 0), n - 1)
    prof_j = min(max(prof_j + dj, 0), n - 1)

    # ---------------------------
    # Influência do professor
    # ---------------------------
    for di in range(-raio_professor, raio_professor + 1):
        for dj in range(-raio_professor, raio_professor + 1):
            ni, nj = prof_i + di, prof_j + dj
            if 0 <= ni < n and 0 <= nj < n:
                nova_grade[ni, nj] = 1
                tau[ni, nj] = 0

    # ---------------------------
    # Regras do autômato
    # ---------------------------
    for i in range(n):
        for j in range(n):
            estado_atual = grade[i, j]
            A = contar_atentos(grade, i, j)

            # Regra 0 → 1
            if estado_atual == 0:
                if A >= theta:
                    nova_grade[i, j] = 1
                    tau[i, j] = 0

            # Regra 1 → 2
            elif estado_atual == 1:
                tau[i, j] += 1
                if tau[i, j] >= Ts:
                    nova_grade[i, j] = 2
                    tau[i, j] = 0

            # Regra 2 → 0
            elif estado_atual == 2:
                tau[i, j] = 0
                if random.random() < pd:
                    nova_grade[i, j] = 0

    # Atualização síncrona
    grade = nova_grade

    # Registrar métricas
    historico_0.append(np.sum(grade == 0))
    historico_1.append(np.sum(grade == 1))
    historico_2.append(np.sum(grade == 2))
    historico_grades.append(grade.copy())
    historico_prof.append((prof_i, prof_j))


# ===============================
# Relatório final
# ===============================

print("Última iteração:")
print("Desatentos:", historico_0[-1])
print("Atentos:", historico_1[-1])
print("Saturados:", historico_2[-1])


In [ ]:
# ===============================
# Visualização estática final
# ===============================

cmap = ListedColormap(['lightgray', 'green', 'red'])

plt.figure(figsize=(6, 6))
plt.imshow(grade, cmap=cmap, vmin=0, vmax=2)
plt.scatter(prof_j, prof_i, c='blue', s=100, label='Professor')
plt.title("Estado final da atenção na turma")
plt.legend()
plt.axis('on')
plt.show()

# ===============================
# Gráfico temporal
# ===============================

plt.figure(figsize=(8, 5))
plt.plot(historico_0, label='Desatenção (0)')
plt.plot(historico_1, label='Atenção (1)')
plt.plot(historico_2, label='Saturação (2)')
plt.xlabel("Iterações (tempo)")
plt.ylabel("Número de estudantes")
plt.title("Evolução temporal dos estados")
plt.legend()
plt.grid(True)
plt.show()

# ===============================
# MAPA DE CALOR DA ATENÇÃO
# ===============================

# Acumulador da atenção ao longo do tempo
mapa_calor_atencao = np.zeros((n, n))

for estado in historico_grades:
    mapa_calor_atencao += (estado == 1)

# Normalizar pelo número de iterações (opcional, recomendado)
mapa_calor_atencao = mapa_calor_atencao / iteracoes

plt.figure(figsize=(7, 6))
plt.imshow(mapa_calor_atencao, cmap='hot')
plt.colorbar(label='Fração de tempo em atenção')
plt.title("Mapa de calor da atenção ao longo da aula")
plt.axis('on')
plt.show()


# ===============================
# MAPA DE CALOR DA TRAJETÓRIA DO PROFESSOR
# ===============================

mapa_calor_professor = np.zeros((n, n))

for (pi, pj) in historico_prof:
    mapa_calor_professor[pi, pj] += 1

# Normalizar pelo número de iterações
mapa_calor_professor = mapa_calor_professor / iteracoes

plt.figure(figsize=(7, 6))
plt.imshow(mapa_calor_professor, cmap='Blues')
plt.colorbar(label='Fração de tempo com presença do professor')
plt.title("Mapa de calor da trajetória do professor")
plt.axis('on')
plt.show()


In [ ]:
# ===============================
# Animação
# ===============================

fig, ax = plt.subplots(figsize=(6, 6))
img = ax.imshow(historico_grades[0], cmap=cmap, vmin=0, vmax=2)
prof_plot = ax.scatter([], [], c='blue', s=100)
ax.axis('on')

def atualizar(frame):
    img.set_array(historico_grades[frame])
    pi, pj = historico_prof[frame]
    prof_plot.set_offsets([[pj, pi]])
    ax.set_title(f"Iteração {frame}")
    return img, prof_plot

ani = animation.FuncAnimation(
    fig,
    atualizar,
    frames=len(historico_grades),
    interval=300,
    repeat=False
)

plt.show()